In [1]:
from copy import deepcopy
from pathlib import Path

import requests

# get the openapi def and update to 3.x
api_original = requests.get(
    "https://converter.swagger.io/api/convert?url=https%3A%2F%2Fjuicesoc.esac.esa.int%2Fdocs%2F%3Fformat%3Dopenapi",
).json()

outfile = Path("../openapi.json")

In [2]:
import re


def forge_id(optype, path):
    params = re.findall("{([^{}]+)}", path)

    for p in params:
        path = path.replace("{" + p + "}", "")

    if "rest_api" in path:
        path = path.replace("rest_api", "")

    path = re.sub(r"/+", "/", path)

    path = path.strip()

    path = path.strip("/")
    path = path.replace("/", "_")
    path = path.replace("-", "_")

    path = optype.lower() + "_" + path
    if len(params) > 0:
        return path + "_by_" + "_and_".join(params)
    return path

In [3]:
api = deepcopy(api_original)
ops = ["get", "post", "put", "delete", "patch"]

known_op_ids = []
known_test_ids = []

for k, item in api["paths"].items():
    print(f"Path {k}")
    for op in ops:
        if op in item.keys():
            it = item.get(op)

            new_op_id = forge_id(op, k)
            print(f"  -> suggested id {new_op_id}")
            if new_op_id in known_test_ids:
                print("  ------------ seems already taken!")
                raise
            else:
                it["operationId"] = new_op_id
                known_test_ids.append(new_op_id)


Path /api-token-auth/
  -> suggested id post_api_token_auth
Path /api-token-refresh/
  -> suggested id post_api_token_refresh
Path /rest_api/checks/
  -> suggested id get_checks
Path /rest_api/checks/{working_group_mnemonic}/
  -> suggested id get_checks_by_working_group_mnemonic
Path /rest_api/checks/{working_group_mnemonic}{format}
  -> suggested id get_checks_by_working_group_mnemonic_and_format
Path /rest_api/checks{format}
  -> suggested id get_checks_by_format
Path /rest_api/config/
  -> suggested id get_config
Path /rest_api/config{format}
  -> suggested id get_config_by_format
Path /rest_api/detailed_scenario/
  -> suggested id get_detailed_scenario
  -> suggested id post_detailed_scenario
Path /rest_api/detailed_scenario/{id}/
  -> suggested id get_detailed_scenario_by_id
Path /rest_api/detailed_scenario/{id}{format}
  -> suggested id get_detailed_scenario_by_id_and_format
Path /rest_api/detailed_scenario{format}
  -> suggested id get_detailed_scenario_by_format
  -> suggested

In [4]:
def find_by_op_id(api, operationId, return_parent_path=False):
    for k, item in api["paths"].items():
        for op in ops:
            if op in item.keys():
                it = item.get(op)
                op_id = it.get("operationId", None)

                if op_id == operationId:
                    if return_parent_path:
                        return item
                    return it

    return None

In [5]:
BODY_PARAMETER_DEFINITION = {
    "name": "body",
    "in": "query",
    "required": True,
    "schema": {"type": "string"},
}


def add_parameter(
    operation_id,
    api,
    definition=BODY_PARAMETER_DEFINITION,
):
    path = find_by_op_id(api, operation_id, return_parent_path=True)
    pars = path.get("parameters", [])

    name = definition["name"]
    for item in pars:
        if name == item["name"]:
            return

    pars.append(definition)

    path["parameters"] = pars
    print("fixed!")

In [6]:
def fix_content_as_array(operation_id, api):
    item = find_by_op_id(api, operation_id)
    try:
        ref = item["responses"]["200"]["content"]["application/json"]["schema"]["$ref"]
    except KeyError:
        return

    newschema = dict(type="array", items={"$ref": ref})

    item["responses"]["200"]["content"]["application/json"]["schema"] = newschema
    print('Fixed as array!')


In [7]:
# FIXES

add_parameter("get_series", api, BODY_PARAMETER_DEFINITION)
add_parameter("get_events",api, BODY_PARAMETER_DEFINITION)
fix_content_as_array( "get_plan", api)

fixed!
fixed!
Fixed as array!


In [8]:
import json

with outfile.open("w") as f:
    json.dump(api, f, indent=4)


In [9]:
# now genrate the module
# Execute the command in the specified directory


import subprocess

command = "poetry run openapi-python-client generate --path openapi.json --overwrite --meta=none --output-path=./src/juice_core_uplink_api_client/".split()

result = subprocess.run(
    command, cwd=outfile.parent, capture_output=True, text=True, check=False
)

# Print the output
print(result.stdout)


Generating src/juice_core_uplink_api_client





